In [1]:
import pandas as pd
%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency
import configparser
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# def init_config():
#     config = configparser.ConfigParser()
#     config.read("../config.ini")
#     return config

# def download_blob(config):
#     """Downloads a blob from the bucket."""
    
#     # Initialize a storage client
#     path = config['gcp']['cloud_storage']
#     df = pd.read_csv(path)
#     return df

# config = init_config()
# df = download_blob(config)

In [2]:
path = "/Users/kaitlin.brabec/OneDrive - PA Consulting Group/computer_recovery/downloads/HR_Analytics.csv.csv"
df = pd.read_csv(path)

In [3]:
categorical_columns = df.select_dtypes(include=['object']).columns
for column in categorical_columns:
    df[column] = df[column].str.replace(' ', '_').str.lower()


In [4]:
# no null
df.isna().any().sum()



np.int64(0)

In [ ]:
# # Store unique values for each int64 column in a dictionary
# unique_values_dict = {}

# for column in df.select_dtypes(include=['int64']).columns:
#     unique_values_dict[column] = df[column].unique()

# # Display the dictionary
# unique_values_dict

In [ ]:
# df.corr(numeric_only=True)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
# import matplotlib.pyplot as plt
# import seaborn as sns
# X = df.drop(columns=['Attrition', "BusinessTravel","OverTime", "Gender", "MaritalStatus", "Department", "EducationField","JobRole" ])
# y = df['Attrition']

# # Standardize the features
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # Apply PCA
# pca = PCA(n_components=2)
# principal_components = pca.fit_transform(X_scaled)

In [ ]:
# pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
# pca_df['Attrition'] = y

In [ ]:
# # Plot the principal components
# plt.figure(figsize=(10, 7))
# sns.scatterplot(x='PC1', y='PC2', hue='Attrition', data=pca_df, palette='Set1', s=100)
# plt.title('PCA of Features Colored by Attrition')
# plt.xlabel('Principal Component 1')
# plt.ylabel('Principal Component 2')
# plt.show()



In [ ]:
# # Explained variance
# explained_variance = pca.explained_variance_ratio_
# explained_variance

In [ ]:


# # PCA components
# components_df = pd.DataFrame(pca.components_, columns=X.columns, index=['PC1', 'PC2'])
# components_df

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# X = df.drop(columns=['Attrition', "BusinessTravel","OverTime", "Gender", "MaritalStatus", "Department", "EducationField","JobRole" ])
# y = df['Attrition']

# # Train a RandomForestClassifier
# clf = RandomForestClassifier(n_estimators=100, random_state=42)
# clf.fit(X, y)

# # Get feature importances
# feature_importances = pd.Series(clf.feature_importances_, index=X.columns)



Feature Engineering

There is an imbalanced problem as there are 1233 No's and 237 Yes in Attrition

In [5]:
numerical_columns = df.select_dtypes(include=['int64'])

In [6]:
def create_and_replace_bins(df, columns, num_bins=5):
    binned_df = df.copy()
    for col in columns:
        col_range = df[col].max() - df[col].min()
        if col_range > 5:
            # Calculate quantile-based bins
            _, bins = pd.qcut(df[col], q=num_bins, duplicates='drop', retbins=True)
            # Adjust bins to integer values
            bins = np.floor(bins).astype(int)
            bins[-1] = bins[-1] + 1  # Ensure the last bin is inclusive
            # Create labels based on bin edges
            labels = [f'{bins[i]}<=x<{bins[i+1]}' for i in range(len(bins)-1)]
            # Apply binning and replace original column
            binned_df[col] = pd.cut(df[col], bins=bins, labels=labels, right=False, include_lowest=True)
    return binned_df

# Apply dynamic binning and replace columns
binned_df_dynamic = create_and_replace_bins(df, numerical_columns, num_bins=5)

# Display the first few rows of the modified dataframe
binned_df_dynamic.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,38<=x<45,yes,travel_rarely,942<=x<1224,sales,1<=x<2,2,life_sciences,1,1<=x<402,...,1,80,0,8<=x<10,0<=x<2,1,5<=x<7,4<=x<7,0<=x<1,4<=x<7
1,45<=x<61,no,travel_frequently,102<=x<391,research_&_development,5<=x<9,1,life_sciences,1,1<=x<402,...,4,80,1,10<=x<17,3<=x<4,3,10<=x<41,7<=x<19,1<=x<4,7<=x<18
2,34<=x<38,yes,travel_rarely,1224<=x<1500,research_&_development,2<=x<5,2,other,1,1<=x<402,...,2,80,0,5<=x<8,3<=x<4,3,0<=x<2,0<=x<1,0<=x<1,0<=x<1
3,29<=x<34,no,travel_frequently,1224<=x<1500,research_&_development,2<=x<5,4,life_sciences,1,1<=x<402,...,3,80,0,8<=x<10,3<=x<4,3,7<=x<10,7<=x<19,1<=x<4,0<=x<1
4,18<=x<29,no,travel_rarely,391<=x<656,research_&_development,2<=x<5,1,medical,1,1<=x<402,...,4,80,1,5<=x<8,3<=x<4,3,2<=x<5,2<=x<4,1<=x<4,2<=x<4


In [7]:
def get_binned_value_counts(df, columns):
    value_counts_dict = {}
    for col in columns:
        value_counts_dict[col] = df[col].value_counts().to_dict()
    return value_counts_dict

# Get value counts for each binned column
get_binned_value_counts(binned_df_dynamic, numerical_columns)


{'Age': {'29<=x<34': 316,
  '45<=x<61': 314,
  '38<=x<45': 308,
  '34<=x<38': 274,
  '18<=x<29': 258},
 'DailyRate': {'391<=x<656': 296,
  '1224<=x<1500': 296,
  '942<=x<1224': 294,
  '102<=x<391': 292,
  '656<=x<942': 292},
 'DistanceFromHome': {'2<=x<5': 359,
  '9<=x<17': 318,
  '17<=x<30': 297,
  '5<=x<9': 288,
  '1<=x<2': 208},
 'Education': {3: 572, 4: 398, 2: 282, 1: 170, 5: 48},
 'EmployeeCount': {1: 1470},
 'EmployeeNumber': {'1654<=x<2069': 295,
  '1<=x<402': 294,
  '402<=x<814': 294,
  '1235<=x<1654': 294,
  '814<=x<1235': 293},
 'EnvironmentSatisfaction': {3: 453, 4: 446, 2: 287, 1: 284},
 'HourlyRate': {'73<=x<87': 310,
  '45<=x<59': 301,
  '87<=x<101': 300,
  '30<=x<45': 282,
  '59<=x<73': 277},
 'JobInvolvement': {3: 868, 2: 375, 4: 144, 1: 83},
 'JobLevel': {1: 543, 2: 534, 3: 218, 4: 106, 5: 69},
 'JobSatisfaction': {4: 459, 3: 442, 1: 289, 2: 280},
 'MonthlyIncome': {'2695<=x<4228': 295,
  '5743<=x<9860': 295,
  '9860<=x<20000': 294,
  '1009<=x<2695': 293,
  '4228<=x<5

In [ ]:
df['Attrition'] = df['Attrition'].map(lambda x : 0 if x == 'no' else 1) 


In [ ]:
df.select_dtypes(include=['object'])

In [ ]:
df["BusinessTravel"].unique()
df["JobRole"].unique()
df["EducationField"].unique()
df["Department"].unique()

In [ ]:
#I would get rid of Marital Status and Gender -- whether or not keeping these featurs is based on the context 
#In this case you want to ensure that decisions are not influenced by gender or marital status, hence promoting fairness and ethics 
# you need to check potential correlation with other features and the target variable

# Encoding categorical variables
df['Gender'] = df['Gender'].map({'male': 1, 'female': 0})
df['MaritalStatus'] = df['MaritalStatus'].map({'single': 0, 'married': 1, 'divorced': 2})

# Contingency tables
contingency_table_gender = pd.crosstab(df['Gender'], df['Attrition'])
contingency_table_marital = pd.crosstab(df['MaritalStatus'], df['Attrition'])

print("Contingency Table - Gender vs Target:")
print(contingency_table_gender)

print("\nContingency Table - Marital Status vs Target:")
print(contingency_table_marital)

# Chi-Square tests
chi2_gender, p_gender, dof_gender, ex_gender = chi2_contingency(contingency_table_gender)
chi2_marital, p_marital, dof_marital, ex_marital = chi2_contingency(contingency_table_marital)

print(f"\nChi-Square Test between gender and target: chi2 = {chi2_gender}, p-value = {p_gender}")
print(f"Chi-Square Test between marital_status and target: chi2 = {chi2_marital}, p-value = {p_marital}")

# Interpretation
alpha = 0.05
print("\nInterpretation:")
if p_gender < alpha:
    print("There is a significant association between gender and target (p < 0.05).")
else:
    print("There is no significant association between gender and target (p >= 0.05).")

if p_marital < alpha:
    print("There is a significant association between marital status and target (p < 0.05).")
else:
    print("There is no significant association between marital status and target (p >= 0.05).")

In [ ]:
df.drop(columns=['Gender'], inplace=True)

In [ ]:
df.drop(columns=['StandardHours', 'Over18', 'EmployeeCount'], inplace=True)

In [ ]:
df.head()

In [ ]:
df.MonthlyIncome.plot.hist()

In [ ]:
sorted_values_mi = df.sort_values(by='MonthlyIncome').reset_index(drop=True)
bins_mi = ['x<2696', '2695<x<4230', '4229<x<5744', '5743<x<9884', '9883<x<19999']
idxs_mi = [int(sorted_values_mi.iloc[(294*i)].MonthlyIncome) if 294*i < len(sorted_values_mi) else int(sorted_values_mi.iloc[len(sorted_values_mi)-1].MonthlyIncome) for i in range(1,6)]
intervals_mi = list(zip(bins_mi,idxs_mi))


In [ ]:
def match_intervals(value):
    for x in intervals_mi:
        if value <= x[1]:
            return x[0]
    return value  # return original value if it doesn't match any interval

df['MonthlyIncome'] = df['MonthlyIncome'].apply(match_intervals)



In [ ]:
df['MonthlyIncome'].value_counts()

In [ ]:
df.MonthlyRate.plot.hist()

In [ ]:
sorted_values_mr = df.sort_values(by='MonthlyRate').reset_index(drop=True)
bins_mr = ['x<6889', '6888<x<11781', '11780<x<16734', '16733<x<21728', '21727<x<26999']
idxs_mr = [int(sorted_values_mr.iloc[(294*i)].MonthlyRate) if 294*i < len(sorted_values_mr) else int(sorted_values_mr.iloc[len(sorted_values_mr)-1].MonthlyRate) for i in range(1,6)]
intervals_mr = list(zip(bins_mr,idxs_mr))
# idxs_mr
# sorted_values_mr.iloc[-1]


In [ ]:
def match_intervals(value):
    for x in intervals_mr:
        if value <= x[1]:
            return x[0]
    return value  # return original value if it doesn't match any interval

df['MonthlyRate'] = df['MonthlyRate'].apply(match_intervals)

In [ ]:
df['MonthlyRate'].value_counts()

In [ ]:
# Plot the histogram
ax = df.NumCompaniesWorked.plot.hist()

# Add labels
for p in ax.patches:
    ax.annotate(str(int(p.get_height())), (p.get_x() * 1.005, p.get_height() * 1.005))

plt.show()

In [ ]:
# Sort values
sorted_values_ncw = df.sort_values(by='NumCompaniesWorked').reset_index(drop=True)

# Calculate the indices for each bin edge
num_bins = 6
idxs_ncw = [int(sorted_values_ncw.iloc[(len(sorted_values_ncw) // num_bins) * i].NumCompaniesWorked) for i in range(1, num_bins)]

# Ensure no duplicate indices
idxs_ncw = sorted(list(set(idxs_ncw)))

# Add min and max values to the bin edges
bin_edges_values = [sorted_values_ncw['NumCompaniesWorked'].min()] + idxs_ncw + [sorted_values_ncw['NumCompaniesWorked'].max()]

# Create bins labels (you can customize these labels)
bins_ncw = [f'x<{bin_edges_values[1]}'] + [f'{bin_edges_values[i]}<x<{bin_edges_values[i+1]}' for i in range(1, len(bin_edges_values)-2)] + [f'x>{bin_edges_values[-2]}']

# Zip bin labels with bin edges for intervals
intervals_ncw = list(zip(bins_ncw, bin_edges_values[1:]))


In [ ]:
# Function to match intervals
def match_intervals(value):
    for x in intervals_ncw:
        if value <= x[1]:
            return x[0]
    return value  # return original value if it doesn't match any interval

# Apply the function to the 'NumCompaniesWorked' column
df['NumCompaniesWorked'] = df['NumCompaniesWorked'].apply(match_intervals)

# Display the updated DataFrame
df['NumCompaniesWorked'].value_counts()

In [ ]:
df.PercentSalaryHike.plot.hist()

In [ ]:
sorted_values_psh = df.sort_values(by='PercentSalaryHike').reset_index(drop=True)
bins_psh = ['x<12', '11<x<13', '12<x<15', '14<x<19', '18<x<25']
idxs_psh = [int(sorted_values_psh.iloc[(294-1)*i].PercentSalaryHike) for i in range(1,6)]
intervals_psh = list(zip(bins_psh,idxs_psh))


In [ ]:
def match_intervals(value):
    for x in intervals_psh:
        if value <= x[1]:
            return x[0]
    return value  # return original value if it doesn't match any interval

df['PercentSalaryHike'] = df['PercentSalaryHike'].apply(match_intervals)

In [ ]:
df['PercentSalaryHike'].value_counts()

In [ ]:
df.TotalWorkingYears.plot.hist()

In [ ]:
sorted_values_twy = df.sort_values(by='TotalWorkingYears').reset_index(drop=True)
bins_twy = ['x<5', '4<x<8', '7<x<10', '9<x<17', '16<x<40']
idxs_twy = [int(sorted_values_twy.iloc[(294*i)].TotalWorkingYears) if 294*i < len(sorted_values_twy) else int(sorted_values_twy.iloc[len(sorted_values_twy)-1].TotalWorkingYears) for i in range(1,6)]
intervals_twy = list(zip(bins_twy,idxs_twy))
intervals_twy

In [ ]:
def match_intervals(value):
    for x in intervals_twy:
        if value <= x[1]:
            return x[0]
    return value  # return original value if it doesn't match any interval

df['TotalWorkingYears'] = df['TotalWorkingYears'].apply(match_intervals)

In [ ]:
df['TotalWorkingYears'].value_counts()

In [ ]:
# Plot the histogram
ax = df.YearsAtCompany.plot.hist()

# Add labels
for p in ax.patches:
    ax.annotate(str(int(p.get_height())), (p.get_x() * 1.005, p.get_height() * 1.005))

plt.show()


In [ ]:
# Age 
df.Age.plot.hist()
# based on the distribution let's split into 5 bins 

In [ ]:
sorted_values = df.sort_values(by='Age').reset_index(drop=True)
bins = ['x<30', '29<x<35', '34<x<39', '44<x<46', '44<x<61']
idxs = [int(sorted_values.iloc[(294-1)*i].Age) for i in range(1,6)]
intervals = list(zip(bins,idxs))



In [ ]:
def match_intervals(value):
    for x in intervals:
        if value <= x[1]:
            return x[0]
    return value  # return original value if it doesn't match any interval

df['Age'] = df['Age'].apply(match_intervals)

In [ ]:
df['Age'].value_counts()

In [ ]:
df.head()

In [ ]:
df.DailyRate.plot.hist()

In [ ]:
sorted_values_dr = df.sort_values(by='DailyRate').reset_index(drop=True)
idxs_dr = [int(sorted_values_dr.iloc[(245*i)].DailyRate) if 245*i < len(sorted_values_dr) else int(sorted_values_dr.iloc[len(sorted_values_dr)-1].DailyRate) for i in range(1,7)]
bins_dr = ['x<335', '334<x<573', '572<x<802', '801<x<1040','1039<x<1274', '1273<x<1499']


intervals_dr = list(zip(bins_dr,idxs_dr))



In [ ]:
def match_intervals(value):
    for x in intervals_dr:
        if value <= x[1]:
            return x[0]
    return value  # return original value if it doesn't match any interval

df['DailyRate'] = df['DailyRate'].apply(match_intervals)

In [ ]:
df['DailyRate'].value_counts()

In [ ]:
df.head()

In [ ]:
df.DistanceFromHome.plot.hist()

In [ ]:
sorted_values_dfh = df.sort_values(by='DistanceFromHome').reset_index(drop=True)
idxs_dfh = [int(sorted_values_dfh.iloc[(294-1)*i].DistanceFromHome) for i in range(1,6)]
bins_dfh = ['x<2', '1<x<5', '4<x<9', '8<x<16','15<x<29']

intervals_dfh = list(zip(bins_dfh,idxs_dfh))


In [ ]:
sorted_values_mi = df.sort_values(by='MonthlyIncome').reset_index(drop=True)
bins_mi = ['x<2561', '2560<x<3633', '3632<x<4930', '4929<x<6538', '6537<x<19999']
idxs_mi = [int(sorted_values_mi.iloc[(294*i)].MonthlyIncome) if 294*i < len(sorted_values_mi) else int(sorted_values_mi.iloc[len(sorted_values_mi)-1].MonthlyIncome) for i in range(1,6)]

intervals_mi = list(zip(bins_mi,idxs_mi))
intervals_mi

In [ ]:

def match_intervals(value):
    for x in intervals_mi:
        if value <= x[1]:
            return x[0]
    return value  # return original value if it doesn't match any interval

df['MonthlyIncome'] = df['MonthlyIncome'].apply(match_intervals)


In [ ]:
df['MonthlyIncome'].value_counts()

In [ ]:
def match_intervals(value):
    for x in intervals_dfh:
        if value <= x[1]:
            return x[0]
    return value  # return original value if it doesn't match any interval

df['DistanceFromHome'] = df['DistanceFromHome'].apply(match_intervals)

In [ ]:
df['DistanceFromHome'].value_counts()

In [ ]:
df.HourlyRate.plot.hist()

In [ ]:
sorted_values_hr = df.sort_values(by='HourlyRate').reset_index(drop=True)
bins_hr = ['x<29', '28<x<34', '33<x<38', '37<x<45', '44<x<61']
idxs_hr = [int(sorted_values_hr.iloc[(294-1)*i].HourlyRate) for i in range(1,6)]
intervals_hr = list(zip(bins_hr,idxs_hr))


In [ ]:
def match_intervals(value):
    for x in intervals_hr:
        if value <= x[1]:
            return x[0]
    return value  # return original value if it doesn't match any interval

df['HourlyRate'] = df['HourlyRate'].apply(match_intervals)

In [ ]:
df['HourlyRate'].unique()

In [ ]:
df['HourlyRate'].value_counts()

In [ ]:
sorted_values_mi = df.sort_values(by='MonthlyIncome').reset_index(drop=True)
bins_mi = ['x<2561', '2560<x<3633', '3632<x<4930', '4929<x<6538', '6537<x<19999']
idxs_mi = [int(sorted_values_mi.iloc[(294*i)].MonthlyIncome) if 294*i < len(sorted_values_mi) else int(sorted_values_mi.iloc[len(sorted_values_mi)-1].MonthlyIncome) for i in range(1,6)]

intervals_mi = list(zip(bins_mi,idxs_mi))
intervals_mi

In [ ]:
def match_intervals(value):
    for x in intervals_mi:
        if value <= x[1]:
            return x[0]
    return value  # return original value if it doesn't match any interval

df['MonthlyIncome'] = df['MonthlyIncome'].apply(match_intervals)

In [ ]:
df['MonthlyIncome'].value_counts()

In [ ]:
# Business Travel

df['BusinessTravel'].value_counts()
# in this case we will use label encoding -- you should check the reason !! we could have choosen one-hot but we didn't ...
df['BusinessTravel'] = df['BusinessTravel'].apply(lambda x: 0 if x =='Non-Travel' else (1 if x == 'Travel_Rarely' else 2))

# Department

df['Department'].value_counts()
department_dummies = pd.get_dummies(df['Department'], prefix='Department', dtype=float) # why now we used one-hot encoding and not label encoding?
df.drop(columns=['Department'], inplace=True)
df = pd.concat([df,department_dummies], axis=1)

# Education Field and JobRole

for field in ['JobRole', 'EducationField', 'Age']:

    df[field].value_counts()
    lb = preprocessing.LabelBinarizer()
    new_data = lb.fit_transform(df[field])
    binary_df = pd.DataFrame(new_data, columns=lb.classes_)
    df.drop(columns=[field], inplace=True)
    pd.concat([df,binary_df], axis=1)


# OverTime

df['OverTime'] = df['OverTime'].map({'Yes': 1, 'No': 0})

In [ ]:
df.head()

update pre-processed data


In [ ]:
BUCKET_URI = "gs://pipeline_hr_analytics/"
FOLDER = "pre_processed_data/"

path = "".join([BUCKET_URI, FOLDER])

df.to_csv(path + 'pre_processed_data.csv', index=False)

auto-ml

In [ ]:
# Will show you how to set up automl 

# First create a dataset 

import os

from google.cloud import aiplatform

PROJECT_ID = "prj-dev-mlcc-flt-01-bbc1"
REGION = "us-west4"

aiplatform.init(project=PROJECT_ID, location=REGION)

dataset = aiplatform.TabularDataset.create(
    display_name="HR Analytics",
    gcs_source=f"{path}pre_processed_data.csv",
)

label_column = "Attrition"

print(dataset.resource_name)
     

In [ ]:
job = aiplatform.AutoMLTabularTrainingJob(
  display_name="train-automl-hr-analytics",
  optimization_prediction_type="classification",
  optimization_objective="maximize-au-prc",
)

model = job.run(
    dataset=dataset,
    target_column=label_column,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=1000,
    model_display_name="test",
    disable_early_stopping=False,
)